In [1]:
import sys
sys.path.append("..")  # Add parent directory to system path

In [9]:
from train_harness.config import LoraConfig, DataConfig
from train_harness.data import InstructionDataset
from datasets import load_dataset


In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
instruction_dataset = InstructionDataset(
    DataConfig(
        instruction_column="instruction",
        input_column="context",
        output_column="response",
    ),
    "databricks/databricks-dolly-15k",
    "mistralai/Mistral-7B-v0.1",
    debug=True,
)

In [52]:
instruction_dataset.process_dataset()


Packing sequences (num_proc=10): 100%|██████████| 15011/15011 [00:01<00:00, 11058.32 examples/s]


In [59]:
processed_ds = instruction_dataset.get_dataset()


In [60]:
instruction_dataset.raw_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [61]:
processed_ds.save_to_disk("data/dolly-15k-packed")


Saving the dataset (0/1 shards):   0%|          | 0/1604 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 179/179 [00:00<00:00, 18531.84 examples/s]


In [63]:
cfg = LoraConfig()


In [64]:
cfg

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [67]:
# mistral playground 

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

In [32]:
new_ds['train'][0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa',
 'input': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a sin